In [64]:
#Load modules necessary

In [ ]:
conda install -c conda-forge pyyaml


In [11]:
conda install spacy

Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.7.0
    latest version: 25.11.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [13]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 74.4 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [14]:
!pip install torch
!pip install transformers
!pip install unbabel-comet
!pip install evaluate



  Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (114 kB)
Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.5
    Uninstalling numpy-2.3.5:
      Successfully uninstalled numpy-2.3.5


In [70]:
import pandas as pd
import numpy as np
import torch
import transformers
import spacy
from comet import download_model, load_from_checkpoint
import evaluate
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [71]:
# LOAD DATA
# ------------------------------------------------
results = pd.read_csv("data/korean_translation.csv")
results.head()

,korean,actual translation,Google translation,Microsoft translation
0,아직 3분 시작된 건 아니죠? 그렇죠?,"My three minutes hasn't started yet, has it?","The 3 minutes haven't started yet, right? Right?","It's not three minutes yet, right? Right?"
1,"크리스 앤더슨:네, 맘대로 시작하실 수 없습니다.","No, you can't start the three minutes.","Chris Anderson: Yes, you can't just start.","Chris Anderson: Yes, you can't start as you want."
2,"3분 다시 설정해주세요, 이건 반칙입니다.","Reset the three minutes, that's just not fair.","Please reset the time to 3 minutes, this is a ...","Please set it again for 3 minutes, this is a f..."
3,"앨리슨 헌트 : 어머나, 여기 참 냉정하네요","Oh my God, it's harsh up here.","Allison Hunt: Oh my, it's so cold in here.","Allison Hunt: Oh my God, it's so cold here"
4,정말이지 긴장되네요,I mean I'm nervous enough as it is.,I'm really nervous.,I'm really nervous


In [74]:
INPUT_CSV = "Data/translations.csv"          # your input file
OUTPUT_CSV = "data/translations_scored.csv"  # file to save with scores

SOURCE_COL = 'korean'
REF_COL = 'actual translation'
MT_COLUMNS = [
    'Google translation',
    "Microsoft translation",
]

In [75]:
BLEURT_MODEL_NAME = "Elron/bleurt-base-512" 

In [76]:
for col in [SOURCE_COL, REF_COL] + MT_COLUMNS:
    if col not in results.columns:
        raise ValueError(f"Column {col} missing. CSV has: {results.columns.tolist()}")

sources    = results[SOURCE_COL].astype(str).tolist()
references = results[REF_COL].astype(str).tolist()

print(f"Loaded {len(results)} rows.")

Loaded 2079 rows.


In [77]:
# SETUP BLEURT
# ------------------------------------------------
print("Loading BLEURT...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bleurt_tokenizer = AutoTokenizer.from_pretrained(BLEURT_MODEL_NAME)
bleurt_model     = AutoModelForSequenceClassification.from_pretrained(BLEURT_MODEL_NAME)
bleurt_model.to(device).eval()

def bleurt_score_batch(refs, cands, batch=16):
    scores = []
    for i in range(0, len(refs), batch):
        with torch.no_grad():
            inputs = bleurt_tokenizer(
                refs[i:i+batch], 
                cands[i:i+batch],
                padding=True, truncation=True, return_tensors="pt"
            ).to(device)
            logits = bleurt_model(**inputs).logits.squeeze(-1)
            scores.extend(logits.cpu().tolist())
    return scores

Loading BLEURT...


In [78]:
print("Loading COMET...")
comet_path  = download_model("Unbabel/wmt22-comet-da")
comet_model = load_from_checkpoint(comet_path)

Loading COMET...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.6.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Encoder model frozen.
/Users/Hawraa_1/opt/anaconda3/envs/py311/lib/python3.11/site-packages/pytorch_lightning/core/saving.py:197: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [79]:
for mt_col in MT_COLUMNS:
    print(f"\n=== Scoring MT column: {mt_col} ===")
    
    mt_output = results[mt_col].astype(str).tolist()

    # --- BLEURT ---
    print("Scoring BLEURT...")
    results[f"{mt_col}_bleurt"] = bleurt_score_batch(references, mt_output)

    # --- COMET ---
    print("Scoring COMET...")
    comet_data = [{"src": s, "mt": m, "ref": r} for s, m, r in zip(sources, mt_output, references)]
    # gpus=0 forces CPU; set gpus=1 if you have a GPU and it's configured
    comet_out  = comet_model.predict(comet_data, batch_size=16, gpus=1)
    
    results[f"{mt_col}_comet"] = comet_out.scores
    print(f"System COMET score for {mt_col}: {comet_out.system_score:.4f}")


=== Scoring MT column: Google translation ===
Scoring BLEURT...


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores


Scoring COMET...


Predicting: 0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISMTOKENIZERS_PARALLELISM=(true | false)
=(true | false)
Predicting DataLoader 0: 100%|██████████| 130/130 [07:35<00:00,  3.50s/it]


System COMET score for Google translation: 0.8338

=== Scoring MT column: Microsoft translation ===
Scoring BLEURT...


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Scoring COMET...


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
Predicting: 0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
TOKENIZERS_PARALLELISM=(true | false)
Predicting DataLoader 0: 100%|██████████| 130/130 [09:58<00:00,  4.60s/it]


System COMET score for Microsoft translation: 0.8203


In [80]:
results.to_csv(OUTPUT_CSV, index=False)
print(f"\nSaved full scored dataset to: {OUTPUT_CSV}")

# Show preview
show_cols = [SOURCE_COL, REF_COL] + MT_COLUMNS + \
            [f"{c}_bleurt" for c in MT_COLUMNS] + \
            [f"{c}_comet"  for c in MT_COLUMNS]

print(results[show_cols].head())


Saved full scored dataset to: data/translations_scored.csv
                        korean  \
0        아직 3분 시작된 건 아니죠? 그렇죠?   
1  크리스 앤더슨:네, 맘대로 시작하실 수 없습니다.   
2      3분 다시 설정해주세요, 이건 반칙입니다.   
3     앨리슨 헌트 : 어머나, 여기 참 냉정하네요   
4                   정말이지 긴장되네요   

                               actual translation  \
0    My three minutes hasn't started yet, has it?   
1          No, you can't start the three minutes.   
2  Reset the three minutes, that's just not fair.   
3                  Oh my God, it's harsh up here.   
4             I mean I'm nervous enough as it is.   

                                  Google translation  \
0   The 3 minutes haven't started yet, right? Right?   
1         Chris Anderson: Yes, you can't just start.   
2  Please reset the time to 3 minutes, this is a ...   
3         Allison Hunt: Oh my, it's so cold in here.   
4                                I'm really nervous.   

                               Microsoft translation  \
0          It's not thr

In [88]:
accuracy_score = pd.read_csv("data/translations_scored.csv")

Google_score =  0.9 * accuracy_score['Google translation_comet'] + 0.1 * accuracy_score['Google translation_bleurt']
Microsoft_score =  0.9 * accuracy_score['Microsoft translation_comet'] + 0.1 * accuracy_score['Microsoft translation_bleurt']
Google_score.mean(),Microsoft_score.mean()

(np.float64(0.7747008360983261), np.float64(0.7571455935619238))

In [86]:
accuracy_score = pd.read_csv("data/translations_scored.csv")
accuracy_score.columns

Index(['korean', 'actual translation', 'Google translation',
       'Microsoft translation', 'Google translation_bleurt',
       'Google translation_comet', 'Microsoft translation_bleurt',
       'Microsoft translation_comet'],
      dtype='object')